#### 3.3. Ending a game
A key concept in computer Go is self-play. In self-play, you usually start with a weak Go-playing agent, have it play against itself, and use the game results to build a stronger bot. In [chapter 4](chapter-4), you’ll use self-play to evaluate board positions. In chapters 9 through 12, you’ll use self-play to evaluate individual moves and the algorithms that selected them.

To take advantage of this technique, you need to make sure your self-play games end. Human games end when neither player can gain an advantage with their next move. This is a tricky concept even for humans. Beginners often end games by playing hopeless moves in the opponent’s territory, or watching their opponent cut into what they believed to be solid territory. For computers, it’s even more difficult. If our bot continues playing as long as legal moves remain available, it’ll eventually fill up its own liberties and lose all of its stones.

You could think up some heuristics to help the bot finish the game in a sensible manner. For example:

Don’t play in a region that’s completely surrounded by stones of the same color.
Don’t play a stone that would have only one liberty.
Always capture an opposing stone with only one liberty.
Unfortunately, all of those rules are too strict. If our bots followed these rules, strong opponents would take advantage to kill groups that should live, rescue groups that should die, or simply gain a better position. Generally, our handcrafted rules should restrict the bot’s options as little as possible, so that more sophisticated algorithms are free to learn the advanced tactics.

To solve this problem, you can look to the history of the game. In ancient times, the winner was simply the player with the most stones on the board. Players would end the game by filling every point they could, leaving only eyes for their groups empty. This could make the end of a game drag out for a long time, so players came up with a way to speed it up. If black clearly controlled a region of the board (any white stone played there would eventually get captured), there was no need for black to fill that region with stones. The players would just agree to count that area for black. This is where the concept of territory came from, and over centuries the rules evolved so that territory was counted explicitly.

Scoring in this method avoids the question of what is territory and what isn’t, but you still have to prevent your bot from killing its own stones; see figure 3.8.